In [100]:
import rasterio
import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Bidirectional
from keras.layers import Conv2D, MaxPooling2D, ConvLSTM2D, TimeDistributed, UpSampling2D, Concatenate
from keras.layers import Activation, BatchNormalization
from keras.optimizers import SGD, Adadelta, Adam
from keras import Input
from keras import Model
import os
import sys
from sklearn.utils import class_weight
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import utilities as util
import importlib
import rnn_tiles
import rnn_pixels
import numpy as np
importlib.reload(rnn_pixels)
importlib.reload(rnn_tiles)

<module 'rnn_tiles' from '/host/Desktop/cnn_dev/florence_mapping/rcnn/rnn_tiles.py'>

In [40]:
util.class_to_index

{11: 0,
 21: 3,
 22: 1,
 23: 1,
 24: 1,
 31: 4,
 41: 2,
 42: 2,
 43: 2,
 52: 2,
 71: 3,
 81: 3,
 82: 3,
 90: 2,
 95: 5}

In [43]:
def strangeBlock(input_block, nb_layers, filters):
    x_list = [input_block]
    c_temp = input_block
    for i in range(nb_layers):
        c_l = ConvLSTM2D(filters=filters,kernel_size=(3,3), activation='elu', padding = 'same', return_sequences=True) (c_temp)
        x_list.append(c_l)
        merge = Concatenate()(x_list)
        c_temp = merge
    return c_temp
    

In [44]:
def strange_network_v2(input_shape, class_count):
    input_img = Input(input_shape)
    strange_block_1 = strangeBlock(input_img, 4, 20)
    pool_1 = TimeDistributed(MaxPooling2D((3,3), padding='same')) (strange_block_1)
    strange_block_2 = strangeBlock(pool_1, 4, 20)
    upsample_1 = TimeDistributed(UpSampling2D((3,3))) (strange_block_2)
    conv_1 = TimeDistributed(Conv2D(32, (3,3)))(upsample_1)
    conv_lc = ConvLSTM2D(filters=class_count,kernel_size=(1,1), activation='softmax', padding = 'same', return_sequences=False, name='landcover') (conv_1)
    reshaping = Reshape((64*64,class_count)) (conv_lc)
    #conv_out_cc = ConvLSTM2D(filters=1,kernel_size=(1,1), padding = 'same', return_sequences=False, name='canopy')(conv_1)
    model = Model(inputs=input_img, outputs=reshaping)  # conv_out_cc])
    return model

In [45]:
def strange_network_v3(input_shape, class_count):
    input_img = Input(input_shape)
    strange_block_1 = strangeBlock(input_img, 4, 20)
    #pool_1 = TimeDistributed(MaxPooling2D((3,3), padding='same')) (strange_block_1)
    strange_block_2 = strangeBlock(strange_block_1, 4, 20)
    conv_lc = ConvLSTM2D(filters=class_count,kernel_size=(1,1), activation='softmax', padding = 'same', return_sequences=False, name='landcover') (strange_block_2)
    reshaping = Reshape((64*64,class_count)) (conv_lc)
    #conv_out_cc = ConvLSTM2D(filters=1,kernel_size=(1,1), padding = 'same', return_sequences=False, name='canopy')(conv_1)
    model = Model(inputs=input_img, outputs=reshaping)  # conv_out_cc])
    return model
    

In [93]:
def strange_network_v4(input_shape, class_count):
    input_img = Input(input_shape)
    strange_block_1 = strangeBlock(input_img, 4, 20)
    #pool_1 = TimeDistributed(MaxPooling2D((3,3), padding='same')) (strange_block_1)
    strange_block_2 = strangeBlock(strange_block_1, 4, 20)
    strange_block_3 = strangeBlock(strange_block_2, 4, 20)
    conv_lc = ConvLSTM2D(filters=class_count,kernel_size=(1,1), activation='softmax', padding = 'same', return_sequences=False, name='landcover') (strange_block_3)
    conv_out_cc = ConvLSTM2D(filters=1,kernel_size=(1,1), activation='sigmoid', padding = 'same', return_sequences=False, name='canopy')(strange_block_3)
    model = Model(inputs=input_img, outputs=[conv_lc, conv_out_cc])
    return model

In [47]:
lc_labels = rasterio.open('/deep_data/recurrent_data/NLCD_DATA/landcover/NLCD_2011_Land_Cover_L48_20190424.img')
canopy_labels = rasterio.open('/deep_data/recurrent_data/NLCD_DATA/canopy/CONUSCartographic_2_8_16/Cartographic/nlcd2011_usfs_conus_canopy_cartographic.img')

In [67]:
tiles = {}
landsat_datasets = {}
tiles['028012'] = ['20110324', '20110612', '20110831', '20111103']
tiles['029011'] = ['20110308', '20110425', '20110831', '20111103']
tiles['028011'] = ['20110308', '20110628', '20110831', '20111103']
for tile_number, dates in tiles.items():
    tile_datasets = []
    l8_image_paths = []
    for date in dates:
        l8_image_paths.append('/deep_data/recurrent_data/tile{}/combined/combined{}.tif'.format(tile_number, date))
    for fp in l8_image_paths:
        tile_datasets.append(rasterio.open(fp))
    landsat_datasets[tile_number] = tile_datasets

In [68]:
tile_size = 64
tile_list = ['028012', '029011', '028011']
class_count = 6
epochs = 200
batch_size = 25

In [10]:
px = rnn_pixels.make_pixels(tile_size, tile_list)

In [11]:
pixels = rnn_pixels.delete_bad_tiles(landsat_datasets,lc_labels, canopy_labels, px, tile_size)

In [12]:
len(pixels)

2481

In [13]:
train_px, val_px, test_px = rnn_pixels.train_val_test_split(pixels, 0.7, 0.7)

train:1736 val:521 test:381


In [94]:
model = strange_network_v4((4, 64, 64, 7), class_count)

In [95]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 4, 64, 64, 7) 0                                            
__________________________________________________________________________________________________
conv_lst_m2d_25 (ConvLSTM2D)    (None, 4, 64, 64, 20 19520       input_4[0][0]                    
__________________________________________________________________________________________________
concatenate_25 (Concatenate)    (None, 4, 64, 64, 27 0           input_4[0][0]                    
                                                                 conv_lst_m2d_25[0][0]            
__________________________________________________________________________________________________
conv_lst_m2d_26 (ConvLSTM2D)    (None, 4, 64, 64, 20 33920       concatenate_25[0][0]             
__________

In [96]:
model = keras.utils.multi_gpu_model(model, 4)

In [101]:
tile_gen = rnn_tiles.rnn_tile_gen(landsat_datasets, lc_labels, canopy_labels, tile_size, class_count)

In [56]:
filepath ='strange_v4_merged.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [57]:
callbacks_list = [checkpoint]

In [58]:
def jaccard_distance(y_true, y_pred, smooth=100):
    """Jaccard distance for semantic segmentation.
    Also known as the intersection-over-union loss.
    This loss is useful when you have unbalanced numbers of pixels within an image
    because it gives all classes equal weight. However, it is not the defacto
    standard for image segmentation.
    For example, assume you are trying to predict if
    each pixel is cat, dog, or background.
    You have 80% background pixels, 10% dog, and 10% cat.
    If the model predicts 100% background
    should it be be 80% right (as with categorical cross entropy)
    or 30% (with this loss)?
    The loss has been modified to have a smooth gradient as it converges on zero.
    This has been shifted so it converges on 0 and is smoothed to avoid exploding
    or disappearing gradient.
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    # Arguments
        y_true: The ground truth tensor.
        y_pred: The predicted tensor
        smooth: Smoothing factor. Default is 100.
    # Returns
        The Jaccard distance between the two tensors.
    # References
        - [What is a good evaluation measure for semantic segmentation?](
           http://www.bmva.org/bmvc/2013/Papers/paper0032/paper0032.pdf)
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [105]:
model.compile(optimizer=Adam(), loss={'landcover':jaccard_distance, 'canopy':'mse'}, metrics={'landcover':['accuracy'], 'canopy':['mae']})

In [108]:
model.fit_generator(generator=tile_gen.tile_generator(train_px, batch_size), 
                    steps_per_epoch=len(train_px) // batch_size, epochs=epochs, verbose=1,
                    validation_data=tile_gen.tile_generator(val_px, batch_size),
                    validation_steps=len(val_px) // batch_size)# callbacks=callbacks_list) #class_weight=weights, callbacks=callbacks_list) 

Epoch 1/200
69/69 [==============================] - 111s 2s/step - loss: 1.3038 - landcover_loss: 0.8613 - canopy_loss: 0.4425 - landcover_acc: 0.6769 - canopy_mean_absolute_error: 0.5065 - val_loss: 1.2803 - val_landcover_loss: 0.8367 - val_canopy_loss: 0.4436 - val_landcover_acc: 0.5890 - val_canopy_mean_absolute_error: 0.5069
Epoch 2/200
69/69 [==============================] - 110s 2s/step - loss: 1.2643 - landcover_loss: 0.8219 - canopy_loss: 0.4425 - landcover_acc: 0.6231 - canopy_mean_absolute_error: 0.5064 - val_loss: 1.2769 - val_landcover_loss: 0.8332 - val_canopy_loss: 0.4438 - val_landcover_acc: 0.5966 - val_canopy_mean_absolute_error: 0.5069
Epoch 3/200
69/69 [==============================] - 110s 2s/step - loss: 1.2644 - landcover_loss: 0.8209 - canopy_loss: 0.4435 - landcover_acc: 0.6230 - canopy_mean_absolute_error: 0.5077 - val_loss: 1.2726 - val_landcover_loss: 0.8290 - val_canopy_loss: 0.4435 - val_landcover_acc: 0.6158 - val_canopy_mean_absolute_error: 0.5068
Epoc

KeyboardInterrupt: 

In [107]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 4, 64, 64, 7) 0                                            
__________________________________________________________________________________________________
lambda_13 (Lambda)              (None, 4, 64, 64, 7) 0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_14 (Lambda)              (None, 4, 64, 64, 7) 0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_15 (Lambda)              (None, 4, 64, 64, 7) 0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_16 

In [ ]:
### model.load_weights('strange_v3_merged.hdf5')

In [109]:
predictions = model.predict_generator(generator = tile_gen.tile_generator(test_px, batch_size=1), steps=len(test_px) // 1, verbose=1)
eval_generator = tile_gen.tile_generator(test_px, batch_size=1)


381/381 [==============================] - 63s 165ms/step


In [110]:
lc = np.asarray(predictions[0])
canopy = np.asarray(predictions[1])

In [115]:
canopy[5]

array([[[0.7310586 ],
        [0.880797  ],
        [0.880797  ],
        ...,
        [0.880797  ],
        [0.880797  ],
        [0.880797  ]],

       [[0.880797  ],
        [0.880797  ],
        [0.95257413],
        ...,
        [0.95257413],
        [0.95257413],
        [0.880797  ]],

       [[0.880797  ],
        [0.95257413],
        [0.95257413],
        ...,
        [0.95257413],
        [0.95257413],
        [0.880797  ]],

       ...,

       [[0.7310586 ],
        [0.7310586 ],
        [0.5       ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.880797  ],
        [0.5       ],
        [0.5       ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.7310586 ],
        [0.        ],
        [0.5       ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]]], dtype=float32)

In [ ]:
pred = np.argmax(predictions,axis=-1)

In [3]:
def strangeBlock(input_block, nb_layers, filters):
    x_list = [input_block]
    c_temp = input_block
    for i in range(nb_layers):
        c_l = Bidirectional(ConvLSTM2D(filters=filters,kernel_size=(3,3), activation='elu', padding = 'same', return_sequences=True)) (c_temp)
        x_list.append(c_l)
        merge = Concatenate()(x_list)
        c_temp = merge
    return c_temp
    

In [ ]:
labels = np.empty(pred.shape)
count = 0
while count < len(labels):
        image_b, label_b = next(eval_generator)
        label_b = np.argmax(label_b, axis=-1)
        labels[count] = label_b
        count += 1


In [ ]:
label_index = labels.reshape(len(test_px)*4096)
pred_index = pred.reshape(len(test_px)*4096)

In [ ]:
pred_index.shape

In [ ]:
importlib.reload(util)

In [ ]:
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
util.plot_confusion_matrix(label_index.astype(np.int), pred_index.astype(np.int), classes=np.array(list(util.indexed_dictionary)),
                      class_dict=util.indexed_dictionary)
# Plot normalized confusion matrix
util.plot_confusion_matrix(label_index.astype(np.int), pred_index.astype(np.int), classes=np.array(list(util.indexed_dictionary)),
                      class_dict=util.indexed_dictionary,
                      normalize=True)
count = 0
for i in range(len(label_index)):
    if(label_index[i] == pred_index[i]):
        count+=1
print("Accuracy is {}".format(count/len(label_index)))

In [ ]:
from sklearn.utils.multiclass import unique_labels


In [ ]:
unique_labels(label_index, pred_index)

In [ ]:
def strange_network(input_shape):
    input_img = Input(input_shape)
    #c_l1 = ConvLSTM2D(filters=32,kernel_size=(2,2), activation='elu', padding = 'same', return_sequences=True) (input_img)
    #c_l2 = ConvLSTM2D(filters=32,kernel_size=(2,2), activation='elu', padding = 'same', return_sequences=True) (c_l1)
    c_l3 = ConvLSTM2D(filters=32,kernel_size=(2,2), activation='elu', padding = 'same', return_sequences=True) (input_img)
    #pool_1 = TimeDistributed(MaxPooling2D((3,3), padding='same')) (c_l3)
    #c_l4 = ConvLSTM2D(filters=32,kernel_size=(2,2), activation='elu', padding = 'same', return_sequences=True) (pool_1)
    #c_l5 = ConvLSTM2D(filters=32,kernel_size=(2,2), activation='elu', padding = 'same', return_sequences=True) (c_l4)
    #c_l6 = ConvLSTM2D(filters=32,kernel_size=(2,2), activation='elu', padding = 'same', return_sequences=True) (pool_1)
    #upsample_1 = TimeDistributed(UpSampling2D((3,3))) (c_l6)
    conv_1 = TimeDistributed(Conv2D(32, (3,3), padding='same'))(c_l3)
    #merge = Concatenate()([c_l3, conv_1])
    conv_lc = ConvLSTM2D(filters=16,kernel_size=(1,1), activation='softmax', padding = 'same', return_sequences=False) (conv_1)
    #conv_out_cc = ConvLSTM2D(filters=1,kernel_size=(1,1), activation='sigmoid', padding = 'same', return_sequences=False)(merge)
    #model = Model(inputs=input_img, outputs=[conv_lc, conv_out_cc])
    model = Model(inputs=input_img, outputs=conv_lc)
    return model

    


In [ ]:
w_tile_gen = rnn_tiles.rnn_tile_gen(landsat_datasets, lc_labels, canopy_labels, tile_size, class_count)
w_generator = w_tile_gen.tile_generator(pixels, batch_size=1)
total_labels = list()
count = 0
while count < len(pixels):
        image_b, label_b = next(w_generator)
        label_b = np.argmax(label_b, axis=-1)
        total_labels.append(label_b)
        count+=1
total_labels = np.asarray(total_labels)
total_labels = total_labels.reshape(len(total_labels)*64*64)
buckets = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0}
for label in total_labels:
    buckets[label] +=1
print(buckets)
weights_list = class_weight.compute_class_weight('balanced', np.unique(total_labels), total_labels)